# Tarea 2

Trabajaré con el conjunto de datos "incidentesviales_noviembre22". Al seguir investigando, descubrí que puedo utilizar la API de OpenStreetMap para complementar la información del camino, incluyendo el límite de velocidad, el número de carriles y el tipo de camino.

En lugar de predecir el número de accidentes por colonia, cambiaré mi enfoque a estimar la probabilidad de que ocurra un accidente en una cierta calle a una cierta hora, basándome en características meteorológicas y características del camino.

Mis objetivos para este tarea son 
1. Remover valores nulos de la columna 'hora'.
2. Corregir el formato de la columna 'fecha'.
3. Truncar la columna 'hora' para remover los minutos (debido a que el clima lo puedo obtener por hora) y agregarlo a la columna 'fecha'
4. Carga de los datos meteorológicos usando la API de Open Meteo y la informacion del camino con la API OpenStreetMap
5. Agrupar los incidentes por calle indicando el numero total de accidentes por calle, temperatura promedio y velocidad maxima permitida en la calle.

## Carga de los datos de incidentes viales
Dado que no utilizaré todas las columnas del archivo, solo cargaré las necesarias para conectar mis datos complementarios. 'fecha' y 'hora' son esenciales para los datos climatológicos, 'folio' servirá como identificador único para cada accidente, y 'longitud' y 'latitud' serán utilizados para obtener información acerca del camino.

In [1]:
import pandas as pd

# Nombre del archivo CSV
archivo_csv = "incidentesviales_noviembre22.csv"

# Especifica las columnas que deseas cargar
columnas_a_cargar = ['fecha', 'hora', 'folio', 'longitud', 'latitud']

# Lee el archivo CSV con solo las columnas especificadas
df_vial = pd.read_csv("../../Datos/" + archivo_csv, usecols=columnas_a_cargar)

# Imprime información básica sobre el DataFrame
print(df_vial.info())
df_vial

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167781 entries, 0 to 167780
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fecha     167781 non-null  object 
 1   hora      167119 non-null  object 
 2   folio     167781 non-null  int64  
 3   longitud  167781 non-null  float64
 4   latitud   167781 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 6.4+ MB
None


,fecha,hora,folio,longitud,latitud
0,01/01/17,0:49,32858,-100.347430,25.740149
1,01/01/17,23:02,32857,-100.407075,25.739637
2,01/01/17,21:48,34349,-100.292420,25.651501
3,01/01/17,8:22,34888,-100.371026,25.728203
4,01/01/17,6:08,34310,-100.387760,25.753216
...,...,...,...,...,...
167776,30/11/22,23:15,103524,-100.352719,25.696954
167777,30/11/22,12:20,103368,-100.307290,25.685117
167778,30/11/22,7:05,103333,-100.273213,25.666169
167779,30/11/22,19:30,103597,-100.374777,25.737189


## Eliminación de valores nulos en la columna 'hora'
Al revisar la estructura del dataframe, observamos que la columna 'hora' contiene 167,119 valores no nulos de un total de 167,781 registros (lo que representa el 0.39% de nuestros datos). Por lo tanto, procedemos a eliminar las filas con valores nulos en la columna 'hora', ya que sin la hora no podemos asignarle un valor meteorológico.

In [2]:
# Elimina las filas donde la columna 'hora' es nula
df_vial.dropna(subset=['hora'], inplace= True)

# Imprime información actualizada del DataFrame
print(df_vial.info())

<class 'pandas.core.frame.DataFrame'>
Index: 167119 entries, 0 to 167780
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   fecha     167119 non-null  object 
 1   hora      167119 non-null  object 
 2   folio     167119 non-null  int64  
 3   longitud  167119 non-null  float64
 4   latitud   167119 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 7.7+ MB
None


## Corregir el formato de la columna 'fecha'
En este paso, abordaremos dos cuestiones relacionadas con la columna 'fecha':

* Detectamos inconsistencias donde en algunos registros el año está representado con 2 dígitos y en otros con 4, por ejemplo, "1/1/2017" y "30/11/22".
* La columna está registrada como una cadena de texto en lugar de estar en formato datetime.

Para solucionar esto, definiremos una función que realizará una división basada en "/", y cuando el elemento del año tenga solo 2 dígitos, agregará "20" para convertirlo en 4 dígitos. Luego, utilizaremos la función pd.to_datetime para convertir la columna al formato datetime.

> **Nota**  
> Como alternativa, podríamos utilizar un ciclo for para iterar a través de los registros y realizar el cambio utilizando df.at[i, 'fecha'], pero demostró ser más lento que el enfoque basado en una función.
>``` python
>for idx, row in df_vial.iterrows():
>    partes_fecha = row['fecha'].split("/")
>    if len(partes_fecha[2]) == 2:  # Si el año tiene solo dos dígitos
>        partes_fecha[2] = "20" + partes_fecha[2]
>    # Actualiza el valor en la fila 'fecha' en el DataFrame
>    df_vial.at[idx, 'fecha'] = "/".join(partes_fecha)
>```

In [3]:
# Define la función para corregir el formato de 'fecha'
def fix_date_format(date_str):
    partes_fecha = date_str.split("/")
    # Si el año tiene solo 2 dígitos, se asume que está en formato YY y se convierte a YYYY
    if len(partes_fecha[2]) == 2:
        partes_fecha[2] = "20" + partes_fecha[2]
    return "/".join(partes_fecha)

# Aplica la función 'fix_date_format' y convierte 'fecha' en datetime
df_vial['fecha'] = pd.to_datetime(df_vial['fecha'].apply(fix_date_format), format='%d/%m/%Y')

df_vial

,fecha,hora,folio,longitud,latitud
0,2017-01-01,0:49,32858,-100.347430,25.740149
1,2017-01-01,23:02,32857,-100.407075,25.739637
2,2017-01-01,21:48,34349,-100.292420,25.651501
3,2017-01-01,8:22,34888,-100.371026,25.728203
4,2017-01-01,6:08,34310,-100.387760,25.753216
...,...,...,...,...,...
167776,2022-11-30,23:15,103524,-100.352719,25.696954
167777,2022-11-30,12:20,103368,-100.307290,25.685117
167778,2022-11-30,7:05,103333,-100.273213,25.666169
167779,2022-11-30,19:30,103597,-100.374777,25.737189


In [4]:
# Combinamos 'fecha' y 'hora' en una nueva columna datetime
df_vial['fecha'] = pd.to_datetime(df_vial['fecha'].astype(str) + ' ' + pd.to_datetime(df_vial['hora'], format='%H:%M').dt.strftime('%H:00:00'))

# Eliminamos la columna 'hora' para evitar duplicación de información
df_vial.drop('hora', axis=1, inplace= True)

df_vial.head()

,fecha,folio,longitud,latitud
0,2017-01-01 00:00:00,32858,-100.347430,25.740149
1,2017-01-01 23:00:00,32857,-100.407075,25.739637
2,2017-01-01 21:00:00,34349,-100.292420,25.651501
3,2017-01-01 08:00:00,34888,-100.371026,25.728203
4,2017-01-01 06:00:00,34310,-100.387760,25.753216


## Carga de los datos meteorologicos
Para obtener estos datos, utilizaremos el sitio web [Open Meteo](https://open-meteo.com/), el cual proporciona una API de acceso gratuito para obtener datos históricos del clima por hora, especificando la latitud y la longitud. En este caso, optaremos por una dirección estática, ya que si lo hiciéramos de manera dinámica y buscáramos información para cada ubicación, excederíamos el límite de llamadas diarias a la API. Por lo tanto, utilizaremos las coordenadas de latitud = 25.68802 y longitud = -100.31482 como referencia para el municipio de Monterrey.


In [5]:
import requests

# Definimos la URL de la API con los parámetros necesarios
api_url = "https://archive-api.open-meteo.com/v1/archive"
latitud = 25.68802
longitud = -100.31482
fecha_inicio = "2017-01-01"
fecha_fin = "2022-11-30"
variables_hora = "temperature_2m,relativehumidity_2m,precipitation,rain,weathercode,windspeed_10m,winddirection_10m,direct_radiation_instant"
zona_horaria = "America/Denver"

# Definimos los parámetros de la consulta
parametros = {
    "latitude": latitud,
    "longitude": longitud,
    "start_date": fecha_inicio,
    "end_date": fecha_fin,
    "hourly": variables_hora,
    "timezone": zona_horaria
}

# Realizamos una solicitud GET a la API
respuesta = requests.get(api_url, params=parametros)

# Verificamos si la solicitud fue exitosa (código de estado 200)
if respuesta.status_code == 200:
    datos = respuesta.json()
    
    df_clima = pd.DataFrame(datos["hourly"])
    
    # Conviertimos la columna 'time' al formato que trabajamos antes
    df_clima['time'] = pd.to_datetime(df_clima['time'], format='%Y-%m-%dT%H:%M').dt.strftime('%Y-%m-%d %H:%M:%S')
    df_clima['time'] = pd.to_datetime(df_clima['time'], format='%Y-%m-%d %H:%M:%S')



else:
    print("Error: No se pudo obtener datos de la API")

# Imprime el DataFrame actualizado
df_clima.head()


,time,temperature_2m,relativehumidity_2m,precipitation,rain,weathercode,windspeed_10m,winddirection_10m,direct_radiation_instant
0,2017-01-01 00:00:00,17.8,78,0.0,0.0,2,5.7,235,0.0
1,2017-01-01 01:00:00,16.8,81,0.0,0.0,2,6.3,246,0.0
2,2017-01-01 02:00:00,16.6,83,0.0,0.0,2,5.9,259,0.0
3,2017-01-01 03:00:00,16.4,83,0.0,0.0,2,6.7,254,0.0
4,2017-01-01 04:00:00,15.9,84,0.0,0.0,2,6.0,245,0.0


## Realizo un merge de los datos meteorológicos
Realizo una operación de left join para agregar información climática a mi tabla de accidentes, utilizando la fecha y la hora como identificadores. Además, como ejercicio, filtro los registros para mostrar únicamente aquellos que ocurrieron cuando la temperatura supera los 39 grados Celsius.

In [6]:
# Combinamos dos DataFrames, df_vial y df_clima, utilizando una combinación izquierda ('left') basada en las columnas 'fecha' y 'time'.
df_new = df_vial.merge(df_clima, left_on='fecha', right_on='time', how='left').drop(columns='time')

# Filtramos las filas del DataFrame resultante df_new donde la columna 'temperature_2m' sea mayor o igual a 39.
df_filtrado = df_new[df_new['temperature_2m'] >= 39]


## Integración de datos de OpenStreetMap

Para enriquecer nuestros registros de accidentes viales, vamos a utilizar OpenStreetMap para obtener información adicional relacionada con las calles o caminos donde ocurrieron los accidentes. Sin embargo, debido a limitaciones de tiempo en esta ocasión solo cargaremos datos de OpenStreetMap para 500 accidentes.

In [7]:
import overpy

# Tomamos las primeras 500 filas del DataFrame df_new y lo almacenamos en una nueva variable df_new.
#df_new = df_new.head(500)

# Crear una instancia de Overpass una vez para reutilizarla en todas las consultas
api = overpy.Overpass()

# Define la función para obtener información de la carretera
def get_road_info(lat, lon):
    # Crear la consulta
    query = f"""
        [out:json];
        (
          way(around:1000, {lat}, {lon})["highway"]["name"]["maxspeed"]["lanes"]["highway"];
        );
        out center;
    """
    
    # Enviar la consulta y obtener el resultado
    result = api.query(query)
    
    # Inicializar variables
    road_name, road_type, lanes, maxspeed = "N/A", "N/A", "N/A", "N/A"
    
    # Procesar el resultado para obtener la información de la carretera
    if result.ways:
        way = result.ways[0]  # Seleccionar la primera carretera encontrada (la más cercana)
        road_name = way.tags.get("name", "N/A")
        road_type = way.tags.get("highway", "N/A")
        lanes = way.tags.get("lanes", "N/A")
        maxspeed = way.tags.get("maxspeed", "N/A")
    
    return road_name, road_type, lanes, maxspeed

# Crear una lista de resultados para df_test
results = df_new.apply(lambda row: get_road_info(row['latitud'], row['longitud']), axis=1)
# Convertir la lista de resultados en un DataFrame con columnas separadas
result_df = pd.DataFrame(results.tolist(), columns=['road_name', 'road_type', 'lanes', 'maxspeed'])

# Concatenar el DataFrame resultante con df_test
df_new = pd.concat([df_new, result_df], axis=1)
df_new


: 

## Creación de grupos de datos
Finalmente, utilizamos una operación de "groupby" para agrupar los incidentes por calle. De esta manera, obtenemos el número de accidentes por calle, el límite de velocidad por calle y la mediana de la temperatura, lo que nos proporciona una perspectiva más completa de la situación.

In [ ]:
# Define las funciones de agregación en un diccionario
aggregations = {  
    'road_name': 'count',         # Contar las filas para obtener cuantos accidentes ocurren por calle
    'temperature_2m': 'median',  # Calcular la mediana de 'temperature_2m'
    'maxspeed': 'max',  # Calcular el máximo de 'maxspeed'
}

# Agrupa por 'road_name' y aplica las funciones de agregación
result = df_new.groupby('road_name').agg(aggregations)

# Renombra las columnas para mayor claridad
result.rename(columns={'road_name': 'total de accidentes', 'maxspeed': 'max_velocidad', 'temperature_2m': 'mediana_temp'}, inplace=True)

# Imprime el resultado
result


,total de accidentes,mediana_temp,max_velocidad
road_name,,,
Antara,1,5.60,20
Avenida Aarón Sáenz Garza,5,12.20,50
Avenida Acapulco,5,18.40,60
Avenida Adolfo Ruíz Cortines,58,17.20,65
Avenida Alfonso Reyes,15,18.30,80
Avenida Benito Juárez,2,21.30,60
Avenida Churubusco,4,18.60,65
Avenida Constitución,7,13.30,80
Avenida Cumbres Élite,12,16.65,40


- Ignorar lo de abajo es codigo para descargar en parte los datos de openstreetmap y guardarlos en un csv

In [9]:
import overpy
import pandas as pd

# Crear una instancia de Overpass una vez para reutilizarla en todas las consultas
api = overpy.Overpass()

# Define la función para obtener información de la carretera
def get_road_info(lat, lon):
    # Crear la consulta
    query = f"""
        [out:json];
        (
          way(around:1000, {lat}, {lon})["highway"]["name"]["maxspeed"]["lanes"]["highway"];
        );
        out center;
    """
    
    # Enviar la consulta y obtener el resultado
    result = api.query(query)
    
    # Inicializar variables
    road_name, road_type, lanes, maxspeed = "N/A", "N/A", "N/A", "N/A"
    
    # Procesar el resultado para obtener la información de la carretera
    if result.ways:
        way = result.ways[0]  # Seleccionar la primera carretera encontrada (la más cercana)
        road_name = way.tags.get("name", "N/A")
        road_type = way.tags.get("highway", "N/A")
        lanes = way.tags.get("lanes", "N/A")
        maxspeed = way.tags.get("maxspeed", "N/A")
    
    return road_name, road_type, lanes, maxspeed

# Número de registros a procesar en cada lote
batch_size = 1000

# Número total de registros en el DataFrame df_new
total_records = len(df_new)

# Inicializar el índice de inicio
start_index = 1000

# Loop para procesar los registros en lotes y actualizar el archivo CSV
while start_index < total_records:
    # Limitar el DataFrame df_new al lote actual
    df_batch = df_new.iloc[start_index:start_index + batch_size]
    
    # Crear una lista de resultados para el lote actual de registros
    results = df_batch.apply(lambda row: get_road_info(row['latitud'], row['longitud']), axis=1)

    # Convertir la lista de resultados en un DataFrame con columnas separadas
    result_df = pd.DataFrame(results.tolist(), columns=['road_name', 'road_type', 'lanes', 'maxspeed'])
    
    # Concatenar el DataFrame resultante con el lote actual de registros de df_new
    df_batch = pd.concat([df_batch, result_df], axis=1)
    
    # Guardar el lote actual en el archivo CSV con la codificación latin1 y sin encabezados si no es la primera vez
    mode = 'w' if start_index == 0 else 'a'
    df_batch.to_csv('carretera_info.csv', index=False, encoding='latin1', mode=mode, header=(start_index == 0))
    
    # Actualizar el índice de inicio para el siguiente lote
    start_index += batch_size

# Mostrar el DataFrame df_new completo con la información de la carretera
print(df_new)


                     fecha   folio    longitud    latitud  temperature_2m  \
0      2017-01-01 00:00:00   32858 -100.347430  25.740149            17.8   
1      2017-01-01 23:00:00   32857 -100.407075  25.739637            18.3   
2      2017-01-01 21:00:00   34349 -100.292420  25.651501            20.9   
3      2017-01-01 08:00:00   34888 -100.371026  25.728203            17.3   
4      2017-01-01 06:00:00   34310 -100.387760  25.753216            15.9   
...                    ...     ...         ...        ...             ...   
167114 2022-11-30 23:00:00  103524 -100.352719  25.696954            11.4   
167115 2022-11-30 12:00:00  103368 -100.307290  25.685117            19.0   
167116 2022-11-30 07:00:00  103333 -100.273213  25.666169            18.5   
167117 2022-11-30 19:00:00  103597 -100.374777  25.737189            13.1   
167118 2022-11-30 16:00:00  103695 -100.407276  25.763334            16.1   

        relativehumidity_2m  precipitation  rain  weathercode  windspeed_10